
WEEK 8 – Diffie-Hellman Key Exchange

In [ ]:
# Diffie-Hellman Key Exchange
def power_mod(base, exponent, modulus):
    return pow(base, exponent, modulus)

p = 23  # prime number
g = 5   # generator

a = 6   # Alice's private key
b = 15  # Bob's private key

A = power_mod(g, a, p)  # Alice's public key
B = power_mod(g, b, p)  # Bob's public key

shared_secret_alice = power_mod(B, a, p)
shared_secret_bob = power_mod(A, b, p)

print("Shared Secret (Alice):", shared_secret_alice)
print("Shared Secret (Bob):", shared_secret_bob)

**WEEK 9 – SHA-256 Hash Function**


In [ ]:
import hashlib

def sha256_hex(msg: bytes) -> str:
    h = hashlib.sha256()
    h.update(msg)
    return h.hexdigest()

messages = [b"", b"abc", b"Hello abc", b"Hai abc"]

for m in messages:
    print(f"Message: {m}")
    print(f"SHA256: {sha256_hex(m)}\n")

WEEK 10 – Digital Signature Standard (DSS/DSA)

In [ ]:
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives import hashes

def generate_keys():
    private_key = dsa.generate_private_key(key_size=1024)
    public_key = private_key.public_key()
    return private_key, public_key

def sign_message(private_key, message):
    return private_key.sign(message, hashes.SHA256())

def verify_signature(public_key, message, signature):
    try:
        public_key.verify(signature, message, hashes.SHA256())
        return True
    except Exception:
        return False

message = b"Hello, this message will be signed."

priv, pub = generate_keys()
signature = sign_message(priv, message)

print("Signature (Hex):", signature.hex())
print("Verification:", verify_signature(pub, message, signature))

WEEK 11 – SSL and Intrusion Detection System (IDS)

Server Code

In [ ]:
import socket, ssl

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)
ssl_context.load_cert_chain(certfile="server.crt", keyfile="server.key")

server_socket.bind(('localhost', 8443))
server_socket.listen(5)
print("SSL Server running...")

with ssl_context.wrap_socket(server_socket, server_side=True) as ssock:
    conn, addr = ssock.accept()
    data = conn.recv(1024).decode()
    print("Received:", data)
    conn.sendall(b"Hello from SSL Server!")
    conn.close()

Client Code

In [ ]:
import socket, ssl

hostname = 'localhost'

# Create an SSL context that ignores certificate verification (for demo)
context = ssl._create_unverified_context()

with socket.create_connection((hostname, 8443)) as sock:
    with context.wrap_socket(sock, server_hostname=hostname) as ssock:
        print("✅ SSL connection established.")
        ssock.sendall(b"Hello from SSL Client!")
        data = ssock.recv(1024).decode()
        print("Received:", data)

generate_cert.py

In [ ]:
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from datetime import datetime, timedelta

# Generate private key
key = rsa.generate_private_key(public_exponent=65537, key_size=2048)

# Write private key to file
with open("server.key", "wb") as f:
    f.write(key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
    ))

# Build self-signed certificate
subject = issuer = x509.Name([
    x509.NameAttribute(NameOID.COUNTRY_NAME, "IN"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, "Telangana"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, "Hyderabad"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, "SSL Demo"),
    x509.NameAttribute(NameOID.COMMON_NAME, "localhost"),
])

cert = (
    x509.CertificateBuilder()
    .subject_name(subject)
    .issuer_name(issuer)
    .public_key(key.public_key())
    .serial_number(x509.random_serial_number())
    .not_valid_before(datetime.utcnow())
    .not_valid_after(datetime.utcnow() + timedelta(days=365))
    .sign(key, hashes.SHA256())
)

# Write certificate to file
with open("server.crt", "wb") as f:
    f.write(cert.public_bytes(serialization.Encoding.PEM))

print("✅ Generated server.crt and server.key successfully!")

Got it! Here is the improved version: **Description** and **Result Analysis** given separately for each week (6–7 lines each) ✅

---

## ✅ WEEK 8 – Diffie-Hellman Key Exchange

### **Description**

Diffie-Hellman is a key exchange algorithm used to generate a shared secret key over an insecure network. It uses a large prime number *p* and a generator *g* to perform modular arithmetic. Each user selects their own private key and derives a public key from it. Public keys are exchanged openly while private keys remain secret. Using the exchanged public key and their own private key, both users compute a common shared secret. The shared secret will be identical for both users if the calculations are correct. This shared secret can then be used for encryption or secure communication.

### **Result Analysis**

In the code, values p = 23 and g = 5 were used to demonstrate the algorithm. Alice selected a private key of 6 and Bob selected 15. They generated their public keys using modular exponentiation and exchanged them. When they calculated the shared secret independently, both obtained the same value: **2**. This confirms that the Diffie-Hellman exchange worked successfully. The result demonstrates secure key generation without directly sending the secret key.

---

## ✅ WEEK 9 – SHA-256 Hash Function

### **Description**

SHA-256 is a cryptographic hash algorithm that produces a fixed 256-bit hash value for any input. It is widely used in digital signatures, authentication, blockchain, and data integrity verification. Hashing is a one-way function, meaning the original input cannot be retrieved from the hash. Even a tiny change in input produces a completely different hash output, showing high sensitivity. It helps ensure that the data has not been modified during transmission. The algorithm is efficient and secure against preimage and collision attacks.

### **Result Analysis**

In the program, four different messages were hashed: an empty string, "abc", "Hello abc", and "Hai abc". Each message produced a unique 64-character hexadecimal hash value. Even though “Hello abc” and “Hai abc” differ only by one character, their hash values are completely different. This shows the avalanche effect of SHA-256. The results confirm that SHA-256 provides a strong guarantee of data integrity and uniqueness for each input message.

---

## ✅ WEEK 10 – Digital Signature Standard (DSA)

### **Description**

DSA is a public-key cryptographic standard used for digital signatures and message authentication. It ensures confidentiality, integrity, and non-repudiation of messages. The sender uses a private key to generate a signature for the message. The receiver uses the sender’s public key to verify whether the message is authentic and unaltered. If the verification fails, it indicates the message has been modified or the sender is not genuine. DSA is commonly used in secure email, authentication systems, and digital certificates.

### **Result Analysis**

The program successfully generates a DSA private and public key pair using 1024-bit security. A message was digitally signed using the private key to produce a unique signature in hexadecimal form. The verification process returned **True**, confirming the signature is valid. This means the message has not been tampered with and is indeed from the legitimate key owner. The experiment demonstrates practical signing and verification functionality. It proves how DSA can defend against message alteration and impersonation attacks.

---

## ✅ WEEK 11 – SSL and Intrusion Detection System (IDS)

### **Description**

SSL (Secure Socket Layer) is a security protocol designed to provide encrypted communication between a client and server. It uses digital certificates to authenticate the server’s identity. The certificate in this program is self-signed and contains details like organization and validity dates. SSL ensures confidentiality, data integrity, and protection against eavesdropping. In real-world systems, SSL is essential for secure websites, online banking, and confidential communication. This setup helps understand basic secure socket creation.

### **Result Analysis**

The server loads the SSL certificate and listens for secure connections on port 8443. The client connects using TLS and successfully establishes an encrypted channel, as shown by "✅ SSL connection established." Both server and client exchange messages securely. The results confirm that SSL encryption is properly implemented and prevents data exposure during transmission. This demonstrates how SSL supports secure communication through authentication and encryption. The overall output verifies that a secure network interaction took place successfully.

---

If you'd like, I can help with:
✅ Adding diagrams
✅ Making this into a formatted report (PDF/Word)
✅ Plagiarism-free wording

Would you like me to convert this into a report format for submission?
